In [2]:
!pip install Dash

In [5]:
!pip install pandas

In [9]:
!pip install datetime

In [10]:
!pip install pytz

In [ ]:
import time
import requests
import pandas as pd
from datetime import datetime
import pytz
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output

# API
Iss_API = 'http://api.open-notify.org/iss-now.json'
update_interval_ms = 5000
trial_length = 100

# fetch iss position
def fetch_iss_position():
    try:
        r = requests.get(Iss_API, timeout=10)
        data = r.json()
        lat = float(data['iss_position']['latitude'])
        lon = float(data['iss_position']['longitude'])
        ts = int(data.get('timestamp', time.time()))
        return lat, lon, ts
    except Exception as e:
        print('Error:', e)
        return None

# Initial figure
fig = go.Figure()

# dash app
app = Dash(__name__)
app.layout = html.Div([
    html.H3("Real Time ISS Tracking"),
    dcc.Graph(id="map", figure=fig, style={"height": "85vh"}),
    dcc.Interval(id="interval", interval=update_interval_ms, n_intervals=0),
    html.Div(id="status", style={"padding": "10px", "fontSize": "14px"})
])

# storing iss trail
trail_df = pd.DataFrame(columns=["ts", "Latitude", "Longitude"])

@app.callback(
    Output("map", "figure"),
    Output("status", "children"),
    Input("interval", "n_intervals")
)
def update_map(n):
    global trail_df, fig
    position = fetch_iss_position()
    if position is None:
        return fig, "Failed to fetch ISS position."
 
    lat, lon, ts = position
    new_row = {"ts": ts, "Latitude": lat, "Longitude": lon}
    trail_df = pd.concat([trail_df, pd.DataFrame([new_row])], ignore_index=True)

    if len(trail_df) > trial_length:
        trail_df = trail_df.iloc[-trial_length:]

    fig = go.Figure()

    # trail
    fig.add_trace(go.Scattergeo(
        lon=trail_df["Longitude"],
        lat=trail_df["Latitude"],
        mode='lines',
        line={"width": 2, "color": "blue"},
        name="Trail"
    ))

    # Current ISS marker
    fig.add_trace(go.Scattergeo(
        lon=[lon],
        lat=[lat],
        mode='markers',
        marker={"size": 10, "color": "red"},
        name="Current"
    ))

    fig.update_geos(
        projection_type="natural earth",
        showcoastlines=True, coastlinecolor="Black",
        showland=True, landcolor="rgb(230,230,230)",
        showocean=True, oceancolor="LightBlue",
    )

    fig.update_layout(title="Real Time ISS Tracking")
    status = f"Last Updated: {time.ctime(ts)}"
    return fig, status

if __name__ == "__main__":
    app.run(debug=True)


C:\Users\JD\AppData\Local\Temp\ipykernel_6632\3154910500.py:55: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



Error: HTTPConnectionPool(host='api.open-notify.org', port=80): Max retries exceeded with url: /iss-now.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x000002AEECBD9970>, 'Connection to api.open-notify.org timed out. (connect timeout=10)'))
Error: HTTPConnectionPool(host='api.open-notify.org', port=80): Max retries exceeded with url: /iss-now.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x000002AEECBD9B80>, 'Connection to api.open-notify.org timed out. (connect timeout=10)'))
Error: HTTPConnectionPool(host='api.open-notify.org', port=80): Max retries exceeded with url: /iss-now.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x000002AEECC12EA0>, 'Connection to api.open-notify.org timed out. (connect timeout=10)'))
Error: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error: HTTPConnectionPool(host='api.open-notify.org', port=80): Max retr